<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2003/03_Tratamiento_de_valores_perdidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Paquetes

In [ ]:
#pip install rpy2 &> /dev/null

In [ ]:
#pip install pandas==1.5.3 &> /dev/null

In [ ]:
#pip install fancyimpute &> /dev/null

### Lectura de datos

Los datos corresponden a la Encuesta Nacional de Hogares del año 2022, en específico de los hogares de la Costa Norte. Seleccionaremos solo algunas columnas de interés:

- RESULT: Resultado Final de la Encuesta
1. Completa
2. Incompleta
3. Rechazo
4. Ausente
5. Vivienda Desocupada
6. No se Inició la Entrevista
7. Otro

- P24A: Las paredes exteriores de la vivienda tienen fachada: Con tarrajeo
1. Total
2. Parcial
3. No está tarrajeada
4. No corresponde

- P24B: Las paredes exteriores de la vivienda tienen fachada:
1. Totalmente
2. Parcialmente
3. Sin pintar

- P101: Tipo de vivienda
1. Casa independiente
2. Departamento en edificio
3. Vivienda en quinta
4. Vivienda en casa de vecindad(callejón solar o corralón)
5. Choza o cabaña
6. Vivienda improvisada
7. Local no destinado para habitación humana
8. Otro

- P102: El material predominante en las paredes exteriores es:
1. Ladrillo o bloque de cemento
2. Piedra o sillar con cal o cemento
3. Adobe
4. Tapia
5. Quincha (caña con barro)
6. Piedra con barro
7. Madera (pona, tornillo, etc)
8. Triplay/calamina/estera
9. Otro material

- P104: ¿Cuántas habitaciones en total tiene la vivienda, sin contar el baño,la cocina, los pasadizos, ni el garage?

- P105A: La vivienda que ocupa su hogar es:
1. Alquilada
2. Propia, totalmente pagada
3. Propia, por invasión
4. Propia, comprándola a plazos
5. Cedida por el centro de trabajo
6. Cedida por otro hogar o institución
7. Otra forma

- P106: ¿Cuánto cree que le pagarían de alquiler mensual (en S/.) ?

- D106: (Deflactado, mensualizado) ¿Cuánto cree que le pagarían de alquiler mensual?

- I106: (Imputado, deflactado, mensualizado) ¿Cuánto cree que le pagarían de alquiler mensual?

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_0 = pd.read_csv('03 - enaho1_costanorte.csv',encoding='latin-1', na_values = [99999,999999,'99999','999999',' ','  ','   '])
df_0.head()

### Preprocesamiento

In [ ]:
df_1 = df_0[((df_0['RESULT'] == 1) | (df_0['RESULT'] == 2))& (df_0['P105A'] != 1)][['P24A','P24B','P101','P102','P104','P105A','P106','D106','I106']].reset_index(drop = True)
df_1.head()

In [ ]:
nuevos_nombres = {'P24A':'PARED_TARRAJEO', 'P24B':'PARED_PINTURA',
                  'P101':'TIPO_VIVIENDA', 'P102': 'MATERIAL_PARED',
                  'P104':'NUMHAB', 'P105A': 'VIVIENDA',
                  'P106':'VALOR_ALQ', 'D106':'VALOR_ALQ_DEF','I106':'VALOR_ALQ_DEF_IMP'}
df_1.rename(columns=nuevos_nombres, inplace=True)
df_1.info()

### Identificación de valores perdidos



#### Exploración

In [ ]:
df_1.isnull().sum()

In [ ]:
col_str = ['PARED_TARRAJEO', 'PARED_PINTURA', 'TIPO_VIVIENDA', 'MATERIAL_PARED','VIVIENDA']
df_1[col_str] = df_1[col_str].astype(str)
df_1.head(20)

In [ ]:
df_1.isnull().sum()

In [ ]:
import numpy as np
df_1 = df_1.replace(['nan', 'null'], np.nan)

In [ ]:
df_1.isnull().sum()

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
msno.bar(df_1, figsize=(15,7), fontsize=12, color='steelblue')
plt.title('Frecuencia de valores completos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.matrix(df_1) # permite ver los missings como espacios en blanco
plt.title('Ubicación de los valores perdidos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.dendrogram(df_1)
plt.title('Valores perdidos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.heatmap(df_1)
plt.title('Asociación de pérdida de valores', fontsize=16)
plt.show()

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R -i df_1

In [ ]:
%R df_1 |> head()

In [ ]:
%R install.packages("naniar")

In [ ]:
%%R
library(naniar)
library(dplyr)
df_1 |> gg_miss_upset()

#### Mecanismos

In [ ]:
df_1[df_1['NUMHAB'].isnull()].describe(include=['object']), df_1[df_1['NUMHAB'].notnull()].describe(include=['object'])

In [ ]:
df_1[df_1['NUMHAB'].isnull()].describe(include=['number']), df_1[df_1['NUMHAB'].notnull()].describe(include=['number'])

In [ ]:
df_1[df_1['VALOR_ALQ_DEF'].isnull()].describe(include=['object']), df_1[df_1['VALOR_ALQ_DEF'].notnull()].describe(include=['object'])

In [ ]:
df_1[df_1['VALOR_ALQ_DEF'].isnull()].describe(include=['number']),df_1[df_1['VALOR_ALQ_DEF'].notnull()].describe(include=['number'])

In [ ]:
%%R
table(as_shadow(df_1)$NUMHAB_NA, df_1$VIVIENDA)

In [ ]:
%%R
table(as_shadow(df_1)$NUMHAB_NA, df_1$VIVIENDA) |> chisq.test()

In [ ]:
%%R
table(as_shadow(df_1)$VALOR_ALQ_DEF_NA, df_1$VIVIENDA)

In [ ]:
%%R
table(as_shadow(df_1)$VALOR_ALQ_DEF_NA, df_1$VIVIENDA) |> chisq.test()

In [ ]:
%%R
table(as_shadow(df_1)$VALOR_ALQ_DEF_NA, df_1$PARED_TARRAJEO)

In [ ]:
%%R
table(as_shadow(df_1)$VALOR_ALQ_DEF_NA, df_1$PARED_TARRAJEO) |> chisq.test()

In [ ]:
%%R
df_1 |> mcar_test()

### Eliminación de datos perdidos

In [ ]:
df_1.dropna(axis=0).info() # axis = 0 elimina filas que tienen al menos un valor perdido (eliminación listwise)

In [ ]:
df_1.dropna(axis=1).info() # axis = 1 elimina columnas que tienen al menos un valor perdido

In [ ]:
df_1.dropna(axis = 0, how='all').info() # Siempre que toda la fila (axis=0) o columna (axis=1) esté ausente

In [ ]:
df_1.dropna(axis = 0, thresh = 3).info() # umbral mínimo de valores completos, debe tener 3 o más valores completos por fila (axis=0) o por columna (axis=1)

In [ ]:
df_1.dropna(axis = 0, thresh = 7).info() # umbral mínimo de valores completos, solo considerará las filas con 13 o más valores completos

### Imputación

#### Hot Deck

In [ ]:
df_1_hotdeck1 = df_1.fillna(method='bfill') # backward fill
df_1_hotdeck1.info()

In [ ]:
df_1_hotdeck1[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_1.iloc[14:21]

In [ ]:
df_1_hotdeck1.iloc[14:21]

In [ ]:
df_1_hotdeck2 = df_1.fillna(method='ffill') # forward fill
df_1_hotdeck2.info()

In [ ]:
df_1_hotdeck2[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_1_hotdeck2.iloc[14:21]

#### Imputación por la media

In [ ]:
df_1.iloc[[15,27, 140, 2746]]# datos originales

In [ ]:
df_1_media = df_1.copy()
df_1_media[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']] = df_1_media[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].fillna(df_1.mean(numeric_only=True))
df_1_media.iloc[[15,27, 140, 2746]] # datos imputados

In [ ]:
df_1_media[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

#### Imputación por la mediana

In [ ]:
df_1.iloc[[15,27, 140, 2746]] # datos originales

In [ ]:
df_1_mediana = df_1.copy()
df_1_mediana[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']] = df_1[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].fillna(df_1.median())
df_1_mediana.iloc[[15,27, 140, 2746]] # datos imputados

In [ ]:
df_1_mediana[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

#### Imputación por la moda

In [ ]:
df_1.iloc[[15,32,4039]] # datos originales

In [ ]:
df_1_moda = df_1.copy()
df_1_moda[['TIPO_VIVIENDA', 'MATERIAL_PARED']] = df_1[['TIPO_VIVIENDA', 'MATERIAL_PARED']].fillna(df_1.mode().iloc[0])
df_1_moda.iloc[[15,32,4039]] # datos imputados

#### Imputación por interpolación





In [ ]:
df_1.iloc[[15,27, 140, 2746]] # datos originales

In [ ]:
df_1_interp1 = df_1.copy()
df_1_interp1['NUMHAB'] = df_1['NUMHAB'].interpolate(method='linear')
df_1_interp1['VALOR_ALQ_DEF'] = df_1['VALOR_ALQ_DEF'].interpolate(method='linear')
df_1_interp1['VALOR_ALQ'] = df_1['VALOR_ALQ'].interpolate(method='linear')
df_1_interp1.iloc[[15,27, 140, 2746]] # con interpolación

In [ ]:
df_1_interp1[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_1_interp2 = df_1.copy()
df_1_interp2['NUMHAB'] = df_1['NUMHAB'].interpolate(method='quadratic')
df_1_interp2['VALOR_ALQ_DEF'] = df_1['VALOR_ALQ_DEF'].interpolate(method='quadratic')
df_1_interp2['VALOR_ALQ'] = df_1['VALOR_ALQ'].interpolate(method='quadratic')
df_1_interp2.iloc[[15,27, 140, 2746]] # con interpolación

In [ ]:
df_1_interp2[['VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']].describe()

#### Imputación por regresión


In [ ]:
df_2 = pd.get_dummies(df_1, columns = ['PARED_TARRAJEO', 'PARED_PINTURA', 'TIPO_VIVIENDA', 'MATERIAL_PARED'])
df_2.info()

In [ ]:
new_columns = [col.replace('.0', '') for col in df_2.columns]
df_2.columns = new_columns
df_2 = df_2.drop('VALOR_ALQ_DEF_IMP', axis=1)
df_2.head()

In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
model   = LinearRegression() # definimos la regresión
imputer = IterativeImputer(estimator=model) # definimos el mecanismo de imputación basado en el modelo de regresión
df_2_regre = pd.DataFrame(imputer.fit_transform(df_2)) # aplicamos el mecanismo de imputación
df_2_regre.columns = df_2.columns # el data frame imputado pasa a tener los mismos nombres de columna que el df original

In [ ]:
df_2_regre.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_regre[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()

#### Imputación por verosimilitud


In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
from fancyimpute import IterativeImputer

imp_vero   = IterativeImputer() # definimos el mecanismo de imputación
df_2_vero = pd.DataFrame(imp_vero.fit_transform(df_2), columns = df_2.columns) # aplicamos el mecanismo de imputación
df_2_vero.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_vero[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()

#### Imputación por KNN


In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

imp_knn = IterativeImputer(estimator = KNeighborsRegressor()) # Define el mecanismo de imputación
imp_knn.fit(df_2) # aplica el mecanismo de imputación
df_2_knn = pd.DataFrame(imp_knn.transform(df_2), columns = df_2.columns) # contruye el data frame con los valores imputados
df_2_knn.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_knn[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()

#### Imputación mediante KNN ponderado

In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

imp_knnw = IterativeImputer(estimator = KNeighborsRegressor(weights='distance')) # Se añade la distancia como peso (más distancia, menos peso, y viceversa)
imp_knnw.fit(df_2)
df_2_knnw = pd.DataFrame(imp_knnw.transform(df_2), columns = df_2.columns)
df_2_knnw.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_knnw[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()

#### Imputación mediante Regresión Ridge Bayesiana

In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
from sklearn.linear_model import BayesianRidge

imp_bridge = IterativeImputer(estimator = BayesianRidge()) ##### esta línea de código es la que cambia respecto a las anteriores
imp_bridge.fit(df_2)
df_2_br = pd.DataFrame(imp_bridge.transform(df_2), columns = df_2.columns)
df_2_br.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_br[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()

#### Multiple Imputation by Chained Equations (MICE)

In [ ]:
df_1.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF','VALOR_ALQ_DEF_IMP']]

In [ ]:
imp_mice = IterativeImputer(random_state=100, max_iter=10)
imp_mice.fit(df_2)
df_2_mice = pd.DataFrame(imp_mice.transform(df_2), columns = df_2.columns)
df_2_mice = imputer.transform(df_2_mice)
df_2_mice = pd.DataFrame(df_2_mice, columns = df_2.columns)
df_2_mice.iloc[[15,27, 32,140, 2746, 4039]][['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']]

In [ ]:
df_1[['VALOR_ALQ_DEF_IMP']].describe()

In [ ]:
df_2_mice[['NUMHAB','VALOR_ALQ','VALOR_ALQ_DEF']].describe()